# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [28]:
from random import randint,choice, choices, random, sample
from dataclasses import dataclass

from copy import copy

import lab9_lib
from collections import defaultdict

#import matplotlib.pyplot as plt

In [29]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

10000100100110001000000101010000001111011110001101: 17.56%
10010110110100111001111110101010001001101011111010: 31.34%
01001011011111101100001110010001100010001111100000: 23.33%
11010010111100101010110011100110111100110101100100: 23.33%
01101100001100000110110010101111011111000100000100: 9.33%
00111101010011000011010011111001010000010010110101: 7.33%
10110111110100011110110111100001010111111011010100: 9.11%
10111111000010001011111000001011110101110010111000: 9.33%
00011000100110010101110110000000011000110101001101: 7.33%
11001101001111110010000100100000100111101100111001: 7.33%
10


EA

In [4]:
fitness = lab9_lib.make_problem(10)

POPULATION_SIZE = 20
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 3
MUTATION_PROBABILITY = 0.2 #0.15?

@dataclass
class Individual:
    fitness: int
    genotype: list[int]

population = [
    Individual(
        genotype=choices([0, 1], k=1000), #k=500?
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]  
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, len(offspring.genotype)-1) 
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, len(ind1.genotype)) 
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    return offspring


for generation in range(1_500): #10_000 first
    offspring = list() 
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:
            p = select_parent(population)
            o = mutate(p)
        else:
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o) 

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring) 
    population.sort(key=lambda i: i.fitness, reverse=True) 
    population = population[:POPULATION_SIZE]
    #print(population[0].fitness)
    print(f"{population[0].fitness:.2%}")


print(fitness.calls)
#reset to 0
fitness._calls = 0

10.98%
10.98%
10.98%
16.49%
16.49%
16.49%
16.49%
16.49%
16.49%
16.49%
16.49%
16.49%
16.49%
16.49%
16.49%
16.49%
16.49%
16.50%
16.50%
16.50%
16.50%
16.51%
16.51%
16.51%
16.51%
16.51%
16.51%
16.51%
16.51%
16.51%
16.51%
16.51%
16.51%
16.51%
16.51%
16.51%
16.52%
16.52%
16.52%
16.53%
16.53%
16.53%
16.53%
16.53%
16.53%
16.53%
16.54%
16.54%
16.54%
16.55%
16.55%
16.55%
16.55%
16.55%
16.55%
16.55%
16.55%
16.55%
16.55%
16.55%
16.56%
16.56%
16.56%
16.56%
16.56%
16.56%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.57%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.58%
16.59%
16.59%
16.59%
16.59%
16.59%
16.59%
16.60%
16.60%
16.60%
16.60%
16.61%
16.62%
16.62%
16.62%
16.62%
16.62%
16.62%
16.62%
16.63%
16.63%
16.63%
16.63%
16.63%
16.63%
16.63%

EA with diversity encouraging through parent selection

This  method calculates the average dissimilarity of each individual in the tournament against all other individuals in the population. It then selects the parent with the lowest average dissimilarity as a measure of high diversity for reproduction

In [31]:
fitness = lab9_lib.make_problem(10)

POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 3
MUTATION_PROBABILITY = 0.2 #0.15?

@dataclass
class Individual:
    fitness: int
    genotype: list[int]

population = [
    Individual(
        genotype=choices([0, 1], k=1000), #k=500?
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)


# Function to calculate average dissimilarity
def calculate_average_dissimilarity(individual, population):
    total_dissimilarity = 0
    for other_individual in population:
        # Calculate dissimilarity by comparing the difference in genotypes
        total_dissimilarity += sum(1 for gene1, gene2 in zip(individual.genotype, other_individual.genotype) if gene1 != gene2)
    # Calculate the average dissimilarity
    return total_dissimilarity / len(population)

# Function for parent selection based on average dissimilarity
def select_parent_with_average_dissimilarity(population):
    tournament_pool = [choice(population) for _ in range(TOURNAMENT_SIZE)]
    
    # Calculate average dissimilarity for each individual in the tournament
    dissimilarity_scores = {id(ind): calculate_average_dissimilarity(ind, population) for ind in tournament_pool}
    
    # Choose the parent with the lowest average dissimilarity
    parent_id = min(dissimilarity_scores, key=dissimilarity_scores.get)
    parent = next(ind for ind in tournament_pool if id(ind) == parent_id)
    return parent







def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, len(offspring.genotype)-1) 
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, len(ind1.genotype)) 
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    return offspring


for generation in range(10_000): #10_000 1_500
    offspring = list() 
    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:
            p = select_parent(population)
            o = mutate(p)
        else:
            p1 = select_parent_with_average_dissimilarity(population)
            p2 = select_parent_with_average_dissimilarity(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o) 

    for i in offspring:
        i.fitness = fitness(i.genotype)
    population.extend(offspring) 
    population.sort(key=lambda i: i.fitness, reverse=True) 
    population = population[:POPULATION_SIZE]
    #print(population[0].fitness)
    print(f"{population[0].fitness:.2%}")


print(fitness.calls)
#reset to 0
fitness._calls = 0

10.59%
11.39%
11.39%
16.19%
16.19%
16.19%
16.19%
16.48%
16.48%
16.48%
16.48%
16.48%
16.48%
16.48%
16.49%
16.49%
16.49%
16.49%
22.19%
22.19%
22.19%
22.20%
22.20%
22.20%
22.20%
22.20%
22.20%
22.20%
22.20%
22.20%
22.20%
22.20%
22.20%
22.21%
22.21%
22.21%
22.21%
22.21%
22.21%
22.21%
22.21%
22.21%
22.21%
22.21%
22.21%
22.21%
22.21%
22.21%
22.21%
22.22%
22.22%
22.22%
22.22%
22.22%
22.22%
22.22%
22.22%
22.22%
22.22%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.23%
22.24%
22.24%
22.24%
22.24%
22.24%
22.24%
22.24%
22.24%
22.24%
22.24%
22.24%
22.24%
22.24%
22.24%
22.24%
22.24%
22.24%
22.25%
22.25%
22.25%
22.25%
22.25%
22.25%
22.25%
22.25%
22.25%
22.25%
22.25%
22.25%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%
22.26%

KeyboardInterrupt: 

EA with Adaptation

The adaptation mechanism aims to address scenarios where fitness improvement stagnates by increasing the mutation probability. This adjustment introduces more genetic diversity into the population, potentially helping the algorithm explore new areas of the search space that might contain better solutions.


In this example, the adapt_mutation_probability function adjusts the mutation probability based on the change in the best fitness value over generations. If the fitness improvement stagnates (less than or equal to a 0.5% increase over 50 generations), it increases the mutation probability by a fixed increment (MUTATION_INCREMENT)

In [ ]:
fitness = lab9_lib.make_problem(10)

POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 3
INITIAL_MUTATION_PROBABILITY = 0.2  # Initial mutation probability
MUTATION_INCREMENT = 0.05  # Amount to increase mutation probability

@dataclass
class Individual:
    fitness: int
    genotype: list[int]

population = [
    Individual(
        genotype=choices([0, 1], k=1000), #k=500?
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual, mutation_prob: float) -> Individual:
    offspring = copy(ind)
    for i in range(len(offspring.genotype)):
        if random() < mutation_prob:
            offspring.genotype[i] = not offspring.genotype[i]
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, len(ind1.genotype))
    offspring = Individual(
        fitness=None,
        genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:]
    )
    return offspring

def adapt_mutation_probability(generations_passed, best_fitness, previous_best_fitness, mutation_prob):
    # Check if fitness has improved by at least 1% in the last 20 generations
    if generations_passed >= 50 and (best_fitness - previous_best_fitness) / previous_best_fitness <= 0.005:
        mutation_prob += MUTATION_INCREMENT  # Increase mutation probability by a fixed increment
    return mutation_prob

# Initialize variables for adaptation
previous_best_fitness = population[0].fitness
mutation_probability = INITIAL_MUTATION_PROBABILITY


for generation in range(10_000):
    offspring = list()
    for counter in range(OFFSPRING_SIZE):
        if random() < mutation_probability:
            p = select_parent(population)
            o = mutate(p, mutation_probability)
        else:
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)

    # Calculate fitness for offspring
    for i in offspring:
        i.fitness = fitness(i.genotype)

    # Sort population by fitness and select the top individuals
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]

    current_best_fitness = population[0].fitness
    mutation_probability = adapt_mutation_probability(generation, current_best_fitness, previous_best_fitness, mutation_probability)

    # Update previous best fitness
    previous_best_fitness = current_best_fitness

    print(f"{population[0].fitness:.2%}")
print(fitness.calls)
fitness._calls = 0

EA with diversity encouraging through BOTH parent selection and adaptation

In [ ]:
from dataclasses import dataclass
from random import randint, choice, random
from copy import copy
import lab9_lib

# Define fitness function
fitness = lab9_lib.make_problem(10)

# Define EA parameters
POPULATION_SIZE = 30
OFFSPRING_SIZE = 20
TOURNAMENT_SIZE = 3
INITIAL_MUTATION_PROBABILITY = 0.2  # Initial mutation probability
MUTATION_INCREMENT = 0.05  # Amount to increase mutation probability

# Define individual data structure
@dataclass
class Individual:
    fitness: int
    genotype: list[int]

# Initialize population with random genotypes and calculate fitness
population = [
    Individual(
        genotype=choices([0, 1], k=1000),
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

for i in population:
    i.fitness = fitness(i.genotype)

# Function to calculate average dissimilarity
def calculate_average_dissimilarity(individual, population):
    total_dissimilarity = 0
    for other_individual in population:
        total_dissimilarity += sum(1 for gene1, gene2 in zip(individual.genotype, other_individual.genotype) if gene1 != gene2)
    return total_dissimilarity / len(population)

# Function for parent selection based on average dissimilarity
def select_parent_with_average_dissimilarity(population):
    tournament_pool = [choice(population) for _ in range(TOURNAMENT_SIZE)]
    dissimilarity_scores = {id(ind): calculate_average_dissimilarity(ind, population) for ind in tournament_pool}
    parent_id = min(dissimilarity_scores, key=dissimilarity_scores.get)
    parent = next(ind for ind in tournament_pool if id(ind) == parent_id)
    return parent

# Mutation function
def mutate(ind: Individual, mutation_prob: float) -> Individual:
    offspring = copy(ind)
    for i in range(len(offspring.genotype)):
        if random() < mutation_prob:
            offspring.genotype[i] = not offspring.genotype[i]
    offspring.fitness = None
    return offspring

# Crossover function
def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, len(ind1.genotype))
    offspring = Individual(
        fitness=None,
        genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:]
    )
    return offspring

# Function to adapt mutation probability based on fitness improvement
def adapt_mutation_probability(generations_passed, best_fitness, previous_best_fitness, mutation_prob):
    if generations_passed >= 50 and (best_fitness - previous_best_fitness) / previous_best_fitness <= 0.005:
        mutation_prob += MUTATION_INCREMENT  # Increase mutation probability by a fixed increment
    return mutation_prob

# Initialize variables for adaptation
previous_best_fitness = population[0].fitness
mutation_probability = INITIAL_MUTATION_PROBABILITY


for generation in range(10_000):
    offspring = list() 
    for counter in range(OFFSPRING_SIZE):
        if random() < mutation_probability:
            p = choice(population)  # Using simple parent selection for mutation
            o = mutate(p, mutation_probability)
        else:
            p1 = select_parent_with_average_dissimilarity(population)
            p2 = select_parent_with_average_dissimilarity(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o) 

    # Calculate fitness for offspring
    for i in offspring:
        i.fitness = fitness(i.genotype)

    # Merge offspring with the current population and select top individuals
    population.extend(offspring) 
    population.sort(key=lambda i: i.fitness, reverse=True) 
    population = population[:POPULATION_SIZE]

    # Adapt mutation probability based on fitness improvement
    current_best_fitness = population[0].fitness
    mutation_probability = adapt_mutation_probability(generation, current_best_fitness, previous_best_fitness, mutation_probability)
    previous_best_fitness = current_best_fitness  # Update previous best fitness for adaptation

    print(f"{population[0].fitness:.2%}")

print(fitness.calls)  # Output the number of fitness function calls
fitness._calls = 0
